# Generate supported H5MD files

In [1]:
import znh5md
import numpy as np
from ase.collections import s22
from ase.build import molecule
from ase.calculators.singlepoint import SinglePointCalculator

In [2]:
!rm -f md.h5
md_io = znh5md.IO("md.h5")
water = molecule("H2O")
for step in range(100):
    water.calc = SinglePointCalculator(water, energy=np.random.rand(), forces=np.random.rand(3,3))
    water.set_velocities(np.random.rand(3, 3))
    md_io.append(water)